In [1]:
import cv2
import threading
import ipywidgets as widgets
from IPython.display import display

In [2]:
button_layout = widgets.Layout(width='250px', height='50px', align_self='center')
output = widgets.Output()
# 退出控件 exit button
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
# 图像控件 Image widget
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))
# 空间布局 spatial distribution
controls_box = widgets.VBox([imgbox, exit_button], layout=widgets.Layout(align_self='center'))

def exit_button_Callback(value):
    global model
    model = 'Exit'
#     with output: print(model)
exit_button.on_click(exit_button_Callback)

# 控制模型状态
model = 'General'


In [3]:
# 构建通道字符串
def build_pipeline(sensor_id):
    width = 720
    height = 640
    framerate = 30
    pipeline = (
        f"nvarguscamerasrc sensor-id={sensor_id} ! "
        f"video/x-raw(memory:NVMM), width={width}, height={height}, format=(string)NV12, framerate=(fraction){framerate}/1 ! "
        "nvvidconv ! "
        "video/x-raw, format=(string)BGRx ! "
        "videoconvert ! "
        "appsink"
    )
    return pipeline

# 图像捕获函数
def capture_frames():
    # 打开GStreamer管道
    cap1 = cv2.VideoCapture(build_pipeline(0), cv2.CAP_GSTREAMER)
    cap2 = cv2.VideoCapture(build_pipeline(1), cv2.CAP_GSTREAMER)
    while True:
        try:
            # 读取帧
            ret1, frame1 = cap1.read()
            ret2, frame2 = cap2.read()
            if (not ret1) and (not ret2):
                print("无法从nvarguscamerasrc读取帧")
                break

            if model == "Exit":
                cv2.destroyAllWindows()
                cap1.release()
                cap2.release()
                break

            # 水平拼接两个图像
            combined_image = cv2.hconcat([frame1, frame2])
            
            imgbox.value = cv2.imencode(".jpg", combined_image)[1].tobytes()

        except KeyboardInterrupt:
            cap1.release()
            cap2.release()


In [ ]:

# 显示控件
display(controls_box,output)

# 在新线程中捕获图像
threading.Thread(target=capture_frames).start()
